In [1]:
import pandas as pd
import numpy as np
import string
import re
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import janitor 
import geopandas as gpd
from shapely.geometry import Point

In [2]:
all_years_311 = pd.read_csv("311_data/all_years_311_202108_202507.csv")

/var/folders/rz/h3gzrv2j09s6d_5vmn0bglw40000gn/T/ipykernel_9586/2277738702.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  all_years_311 = pd.read_csv("311_data/all_years_311_202108_202507.csv")


In [8]:
all_years_311[all_years_311.duplicated(subset= ["date_requested", "Date/Time Opened", "public_description", "street_address", "lat", "lng"])]


,service_request_id,sap_notification_number,date_requested,case_age_days,case_record_type,date_closed,status,lat,lng,street_address,...,clean_text_for_sentiment,misspelled_words,num_misspelled,year,month,day,date,time,time_period,week_type
4477,3503365,NaN,2021-09-12,4,Neighborhood Policing,2021-09-15 00:00:00,Closed,32.715170,-117.156620,950 9th Ave,...,The same group is there every day in and out d...,[],0,2021,9,12,2021-09-12,1,Dark,Weekend
6295,3528546,NaN,2021-10-04,0,Neighborhood Policing,2021-10-04 00:00:00,Closed,32.715539,-117.166516,"989 State St, San Diego, CA 92101, USA",...,test by panda ! do not enforce,[],0,2021,10,4,2021-10-04,1,Dark,Weekday
6928,3526496,NaN,2021-10-01,6,Neighborhood Policing,2021-10-07 00:00:00,Closed,32.731705,-117.158500,664?684 El Prado,...,Transients /mentally ill/ addicts,[],0,2021,10,1,2021-10-01,1,Dark,Weekday
6931,3526497,NaN,2021-10-01,6,Neighborhood Policing,2021-10-07 00:00:00,Closed,32.731705,-117.158500,664?684 El Prado,...,Transients /mentally ill/ addicts,[],0,2021,10,1,2021-10-01,1,Dark,Weekday
7899,3548753,NaN,2021-10-21,7,Neighborhood Policing,2021-10-28 00:00:00,Referred,32.712816,-117.148197,707 17th St,...,Trash & Homeless,[],0,2021,10,21,2021-10-21,1,Dark,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212338,5302395,NaN,2025-07-22,2,Neighborhood Policing,7/24/2025,Closed,32.713101,-117.157127,"733 Eighth Ave, San Diego, CA 92101, USA",...,A few people work in shelter let other people ...,"['inso', 'outsidemake', 'methank']",3,2025,7,22,2025-07-22,1,Dark,Weekday
212345,5302697,NaN,2025-07-22,2,Neighborhood Policing,7/24/2025,Referred,32.713101,-117.157127,"733 Eighth Ave, San Diego, CA 92101, USA",...,Recently a few people from shelter making pois...,"['theatersrestaurantslibrariesplease', 'places...",3,2025,7,22,2025-07-22,1,Dark,Weekday
212346,5302701,NaN,2025-07-22,2,Neighborhood Policing,7/24/2025,Referred,32.713101,-117.157127,"733 Eighth Ave, San Diego, CA 92101, USA",...,Recently a few people from shelter making pois...,"['theatersrestaurantslibrariesplease', 'places...",3,2025,7,22,2025-07-22,1,Dark,Weekday
212981,5311137,NaN,2025-07-29,4,Neighborhood Policing,8/2/2025,Closed,32.750268,-117.215113,3249 Fordham St,...,HOMELESS ENCAMPMENT,[],0,2025,7,29,2025-07-29,1,Dark,Weekday


In [7]:
all_years_311.columns

Index(['service_request_id', 'sap_notification_number', 'date_requested',
       'case_age_days', 'case_record_type', 'date_closed', 'status', 'lat',
       'lng', 'street_address', 'zipcode', 'council_district',
       'comm_plan_code', 'comm_plan_name', 'park_name', 'case_origin',
       'referred', 'iamfloc', 'floc', 'public_description', 'Case Number',
       'Department', 'Date/Time Opened', 'Status', 'Case Reason',
       'Display Referral Information', 'prop_uppercase', 'prop_all_caps',
       'num_exclamation', 'num_question', 'clean_text', 'word_count',
       'clean_text_for_sentiment', 'misspelled_words', 'num_misspelled',
       'year', 'month', 'day', 'date', 'time', 'time_period', 'week_type'],
      dtype='object')

In [3]:
all_years_311.head()

,service_request_id,sap_notification_number,date_requested,case_age_days,case_record_type,date_closed,status,lat,lng,street_address,...,clean_text_for_sentiment,misspelled_words,num_misspelled,year,month,day,date,time,time_period,week_type
0,3454060,NaN,2021-08-01,12,Neighborhood Policing,2021-08-12 00:00:00,Closed,32.696611,-117.114868,"922 S 37th St, San Diego, CA 92113, USA",...,A MAN LIVES IN THE TRUCK AND RUNS A BUSINESS B...,"['matresses', 'onece']",2,2021,8,1,2021-08-01,1,Dark,Weekend
1,3454993,NaN,2021-08-02,1,Neighborhood Policing,2021-08-03 00:00:00,Closed,32.762736,-117.066252,6106 Adams Ave,...,HOMELESS CAMPING ON CUL DE SAC BEHIND SHOPPING...,['cul'],1,2021,8,2,2021-08-02,1,Dark,Weekday
2,3457547,NaN,2021-08-04,40,Neighborhood Policing,2021-09-13 00:00:00,Referred,32.788346,-117.254238,3947?3951 Strand Way,...,Trailer parked for several days,[],0,2021,8,4,2021-08-04,1,Dark,Weekday
3,3456172,NaN,2021-08-03,30,ESD Complaint/Report,2021-09-02 00:00:00,Closed,32.752666,-117.146516,4130 Park Blvd,...,Carts full of junk on Park Boulevard behind ch...,[],0,2021,8,3,2021-08-03,1,Dark,Weekday
4,3457550,NaN,2021-08-04,19,ESD Complaint/Report,2021-08-23 00:00:00,Closed,32.746003,-117.159906,525 Evans Place,...,Morning Encampment,[],0,2021,8,4,2021-08-04,1,Dark,Weekday


In [4]:
all_years_311[ (all_years_311["year"] == 2025)]

,service_request_id,sap_notification_number,date_requested,case_age_days,case_record_type,date_closed,status,lat,lng,street_address,...,clean_text_for_sentiment,misspelled_words,num_misspelled,year,month,day,date,time,time_period,week_type
189919,5055744,NaN,2025-01-01,0,Neighborhood Policing,1/1/2025,Referred,32.569270,-117.066668,"3120 Iris Ave, San Ysidro, CA 92173, USA",...,RP reported seeing smoke across from the troll...,"['sdfd', 'rp', 'utl', 'sdpd']",4,2025,1,1,2025-01-01,1,Dark,Weekday
189920,5055768,NaN,2025-01-01,0,Neighborhood Policing,1/1/2025,Closed,32.759632,-117.195942,0554 Taylor Street,...,Please either: close this park at Serra Palm s...,"['grafitti', 'fetanyl', 'vandalsim', 'preisido...",6,2025,1,1,2025-01-01,1,Dark,Weekday
189921,5055787,NaN,2025-01-01,0,Neighborhood Policing,1/1/2025,Closed,32.834699,-117.273876,701?799 Fern Glen,...,Person dwelling in vehicle next to La Jolla Hi...,['jolla'],1,2025,1,1,2025-01-01,1,Dark,Weekday
189922,5055799,NaN,2025-01-01,0,Neighborhood Policing,1/1/2025,Closed,32.834689,-117.273873,701?799 Fern Glen,...,Car parked more than 72 hours. White explorer....,['jolla'],1,2025,1,1,2025-01-01,1,Dark,Weekday
189923,5055827,NaN,2025-01-01,0,Neighborhood Policing,1/1/2025,Closed,32.750050,-117.161729,330 Washington St,...,Encampment blocking sidewalk again,[],0,2025,1,1,2025-01-01,1,Dark,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213944,5261882,NaN,2025-06-20,136,Neighborhood Policing,11/3/2025,Closed,32.706480,-117.119907,Francis St,...,How long are we going to let these people camp...,['beennyears'],1,2025,6,20,2025-06-20,1,Dark,Weekday
213945,5109198,NaN,2025-02-13,266,Neighborhood Policing,11/6/2025,Closed,32.697381,-117.136405,2317?2365 Kearney Ave,...,"Parked over 72hrs, Living in motorhome 25 ft f...","['fecesoil', 'useplease', 'ft', '72hrs', 'moto...",5,2025,2,13,2025-02-13,1,Dark,Weekday
213946,5121852,NaN,2025-02-24,255,Neighborhood Policing,11/6/2025,Closed,32.728671,-117.065395,"6144 Federal Blvd, San Diego, CA 92114, USA",...,Vehicle has been parked here for over a week,[],0,2025,2,24,2025-02-24,1,Dark,Weekday
213947,5169378,NaN,2025-04-02,218,Neighborhood Policing,11/6/2025,Closed,32.796200,-117.209677,"2525 Mission Bay Dr., San Diego, CA 92109, USA",...,various motorhomes parked along mission bay dr...,['motorhomes'],1,2025,4,2,2025-04-02,1,Dark,Weekday


In [9]:
# create point geometries using coordinates
geometry = [Point(xy) for xy in zip(all_years_311['lng'], all_years_311['lat'])]

In [10]:
# convert to geodataframe

all_years_311_gdf = gpd.GeoDataFrame(all_years_311, geometry = geometry, crs="EPSG:4326")

# reproject to California Zone 6
all_years_311_gdf= all_years_311_gdf.to_crs("EPSG:2230")


In [11]:
all_years_311_gdf.head()

,service_request_id,sap_notification_number,date_requested,case_age_days,case_record_type,date_closed,status,lat,lng,street_address,...,misspelled_words,num_misspelled,year,month,day,date,time,time_period,week_type,geometry
0,3454060,NaN,2021-08-01,12,Neighborhood Policing,2021-08-12 00:00:00,Closed,32.696611,-117.114868,"922 S 37th St, San Diego, CA 92113, USA",...,"['matresses', 'onece']",2,2021,8,1,2021-08-01,1,Dark,Weekend,POINT (6295589.28 1834343.228)
1,3454993,NaN,2021-08-02,1,Neighborhood Policing,2021-08-03 00:00:00,Closed,32.762736,-117.066252,6106 Adams Ave,...,['cul'],1,2021,8,2,2021-08-02,1,Dark,Weekday,POINT (6310734.246 1858281.286)
2,3457547,NaN,2021-08-04,40,Neighborhood Policing,2021-09-13 00:00:00,Referred,32.788346,-117.254238,3947?3951 Strand Way,...,[],0,2021,8,4,2021-08-04,1,Dark,Weekday,POINT (6253034.98 1868103.453)
3,3456172,NaN,2021-08-03,30,ESD Complaint/Report,2021-09-02 00:00:00,Closed,32.752666,-117.146516,4130 Park Blvd,...,[],0,2021,8,3,2021-08-03,1,Dark,Weekday,POINT (6286028.505 1854820.184)
4,3457550,NaN,2021-08-04,19,ESD Complaint/Report,2021-08-23 00:00:00,Closed,32.746003,-117.159906,525 Evans Place,...,[],0,2021,8,4,2021-08-04,1,Dark,Weekday,POINT (6281890.694 1852431.713)


In [16]:
all_years_311_gdf = all_years_311_gdf.drop(columns="Status")

In [17]:
all_years_311_gdf.to_file("311_data/all_years_311_points.gpkg")